In [ ]:
%matplotlib
import csv
import sklearn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import mpl_toolkits.mplot3d.axes3d as p3
import matplotlib.animation as animation
import scipy

In [ ]:
df = pd.read_csv('data_GP/data_GP/AG/block1-UNWEIGHTED-SLOW-NONDOMINANT-RANDOM/20161213203046-59968-right-speed_0.500.csv', delimiter=',')

In [ ]:
##### create array of pairs of all of these
frames = np.zeros((len(df), 50, 4))
times = np.zeros(len(df))

for i, row in df.iterrows():
    for count, joint_idx in enumerate(range(11,52*4,4)):
#         print(frames[i,4+count,:], i, count)
        frames[i,count,:] = row[joint_idx:joint_idx+4]
#     for count, joint_idx in enumerate(range(0,3*3,3)):
#         frames[i,count,:] = row[2+joint_idx:2+joint_idx+3]
    times[i] = row[1]

In [ ]:
frames[0,bone[0],0]

In [ ]:
print(frames.shape)

In [ ]:
bone_list = [[39,40],[39,42],[38,39],[37,38],[36,37], \
             [46,49],[46,47],[45,46],[44,45],[43,44], \
             [36,43],[30,43],[30,36],[30,31],[31,32],[32,33], \
             [33,34],[33,35],[34,44],[35,37],[32,6],[32,18], \
             [18,19],[6,7], \
             [31,7],[7,8],[8,9],[9,15],[14,15],[15,10],[15,11],[15,12],[15,13],[15,16],[16,17], \
             [31,19],[19,20],[20,21],[21,27],[27,26],[26,22],[27,23],[27,24],[27,25],[27,28],[28,29], \
             [31,1],[1,0],[1,2],[1,3],[3,4],[4,5], [32,1]]
fig = plt.figure()
ax = p3.Axes3D(fig)
ax.set_xlim3d([0,1])

# ax.set_xlim3d([-1,1])
ax.set_xlabel('X')
ax.set_ylim3d([0,1])

# ax.set_ylim3d([2,3])
ax.set_ylabel('Y')

ax.set_zlim3d([0,1])

# ax.set_zlim3d([-1,1])
ax.set_zlabel('Z')

data = np.array((normalize(frames[0,:,2]), \
           normalize(frames[0,:,0]), \
           normalize(frames[0,:,1])))


line_data = []
for it in range(0,1030,2):
#     ax = p3.Axes3D(fig)
    ax.set_xlim3d([0,1])
    ax.set_xlabel('X')
    ax.set_ylim3d([0,1])
    ax.set_ylabel('Y')
    ax.set_zlim3d([0,1])
    ax.set_zlabel('Z')
    data = np.array((normalize(frames[it,:,2]), \
           normalize(frames[it,:,0]), \
           normalize(frames[it,:,1])))
    for i, bone in enumerate(bone_list):
        if frames[it,bone[0],3] <= 0 or frames[it,bone[1],3] <= -1:
            continue
        x,y,z = [np.array( [ data[j,bone[0]], data[j, bone[1]] ] ) for j in range(3)]
        ax.plot(x,y,z)
#     ax.scatter(normalize(frames[it,:,2]), \
#        normalize(frames[it,:,0]), \
#        normalize(frames[it,:,1]) )
    ax.scatter(normalize(frames[it,:,2])[20], \
       normalize(frames[it,:,0])[20], \
       normalize(frames[it,:,1])[20] )
    plt.savefig('anim/test{}.png'.format(it))
    ax.clear()

print(data[0].shape)

In [ ]:
def normalize(X):
    a = X - np.nanmin(X)
    d = np.nanmax(X) - np.nanmin(X)
    return a/d

def update_anim(i, frames, graph):
    data = (normalize(frames[i,10:20,2]), \
           normalize(frames[i,10:20,0]), \
           normalize(frames[i,10:20,1]) )
    graph._offsets3d = data

fig = plt.figure()
ax = p3.Axes3D(fig)




ax.set_xlim3d([0,1])
ax.set_xlabel('X')

ax.set_ylim3d([0,1])
ax.set_ylabel('Y')

ax.set_zlim3d([0,1])
ax.set_zlabel('Z')

i=0
graph = ax.scatter(normalize(frames[i,:,2]), \
       normalize(frames[i,:,0]), \
       normalize(frames[i,:,1]) )

frame_nums = np.arange(100)
line_ani = animation.FuncAnimation(fig, update_anim, len(frames), fargs=(frames, graph),
                                   interval=50, blit=False, repeat=False)

In [ ]:
startWin = np.arange(0, len(df), 10)
endWin = np.arange(9, len(df), 10)
data = frames

In [ ]:
from scipy.spatial.distance import cdist, squareform

def rbf_kernel(x1, x2, l=1.0, sigma=1.0, noise=0.0):
    pairwise_dists = (cdist(x1,x2, 'euclidean')) 
    K = sigma**2 * np.exp((-1/(2*l**2)) * pairwise_dists **2) + noise**2 * np.eye(len(x1))
    return K
def matern_kernel(x1, x2, l=1, sigma=1.5):
    pairwise_dists = squareform(cdist(x1,x2, 'euclidean'))
    
    return (1.0 + np.sqrt(3.0) * pairwise_dists) * np.exp(-np.sqrt(3) * pairwise_dists)

class GP:
    # kernel is a function to be called
    def __init__(self, kernel=None):
        self.kernel = kernel
        
    def objective_function(self, theta):
#         K = self.kernel(self.X, self.X, l=theta[0], sigma=theta[1]) + self.noise_level**2 * np.eye(len(self.X))
        K = self.kernel(self.X, self.X, l=theta[0], sigma=theta[1], noise=theta[2])
        try:
            L = scipy.linalg.cholesky(K, lower=True)
        except:
            return np.inf
        beta = np.linalg.solve(L.T, np.linalg.solve(L,self.X))
        nll = 0.5*self.y.T@beta + np.sum(np.log(np.diagonal(L))) + 0.5*len(self.X)*np.log(2*np.pi)
        return nll
    
    def fit(self, X, y, noise_level=.0005):
        self.noise_level = noise_level
        self.X = X
        self.y = y
        # Using algorithm 2.1 from Rasmussen book. Uses Cholesky factorization
        bounds = ((0, None), (0, None), (0,1))
        init_theta = [1.0, 1.0, .5]
#         opt = scipy.optimize.minimize(self.objective_function, init_theta, method='L-BFGS-B', bounds=bounds)
        opt,val,d = scipy.optimize.fmin_l_bfgs_b(self.objective_function, init_theta, bounds=bounds, approx_grad=True)
        self.opt_l, self.opt_sigma, self.opt_noise = opt
        print(opt)
        print(val)
        print(d)
        
#         K = self.kernel(self.X, self.X, l=self.opt_l, sigma=self.opt_sigma) + self.noise_level * np.eye(len(self.X))
        K = self.kernel(self.X, self.X, l=self.opt_l, sigma=self.opt_sigma, noise=self.opt_noise)

        self.L = scipy.linalg.cholesky(K, lower=True)
#         self.alpha = scipy.linalg.cho_solve((self.L, True), self.y)
        self.alpha = np.linalg.solve(self.L.T, np.linalg.solve(self.L,self.X))

        return self.opt_l, self.opt_sigma
    
    
    def predict(self, X):
#         K = self.kernel(self.X, self.X, l=self.opt_l, sigma=self.opt_sigma) + self.noise_level * np.eye(len(self.X))
        K = self.kernel(self.X, self.X, l=self.opt_l, sigma=self.opt_sigma, noise=self.opt_noise)

        Kstar = self.kernel(X, self.X, l=self.opt_l, sigma=self.opt_sigma)
#         K2star = self.kernel(X, X, l=self.opt_l, sigma=self.opt_sigma) + self.noise_level * np.eye(len(X))
        K2star = self.kernel(X, X, l=self.opt_l, sigma=self.opt_sigma)

        inverse = np.linalg.inv(K)
        print("kstar shape", Kstar.shape)
        
#         mu = Kstar @ inverse @ self.y
        mu = Kstar.T @ self.alpha
        v = np.linalg.solve(self.L, Kstar)
        var = K2star - v.T @ v

        
        return mu, np.sqrt(var), cov
        
        

In [ ]:
y = data[startWin[0]:endWin[0], 0, 0]
X = times[startWin[0]:endWin[0]].reshape(-1, 1)
gp = GP(kernel=rbf_kernel)
gp.fit(X,y, noise_level=.001)
mu, std, _ = gp.predict(X)
print(y)
print(mu)
print((y-mu))
print(std)


In [ ]:
# y = data[startWin[0]:endWin[0], 0, 0]
# X = times[startWin[0]:endWin[0]].reshape(-1, 1)
print(endWin[4])
between = np.random.randint(endWin[4], size=30)
# between = [0, 10, 20, 30, 40, 48]
# between = range(startWin[0],endWin[4])
X = np.array([times[i] for i in between]).reshape(-1, 1)
y = np.array([data[i, 0, 0] for i in between])
print(y.shape)
Xtest = times[startWin[0]:endWin[4]].reshape(-1, 1)
print(y.shape)
gp = GP(kernel=rbf_kernel)
gp.fit(X,y, noise_level=0.005)
mu, std, _ = gp.predict(Xtest)
print(y)
print(mu)
# print((y-mu))
# print(cov)

plt.plot(X, y, 'bs', ms=8)
# plt.plot(X, f_post)
plt.gca().fill_between(Xtest.flat, mu-2*std, mu+2*std, color="#dddddd")
plt.plot(Xtest, mu, 'r--', lw=2)
# plt.axis([-1, 1, -1, 1])
plt.title('Three samples from the GP posterior')
plt.show()

In [ ]:
# y = data[startWin[0]:endWin[0], 0, 0]
# X = times[startWin[0]:endWin[0]].reshape(-1, 1)
print(endWin[4])
between = np.random.randint(endWin[4], size=30)
# between = [0, 10, 20, 30, 40, 48]
X = np.array([times[i] for i in between]).reshape(-1, 1)
y = np.array([data[i, 0, 0] for i in between])
print(y.shape)
Xtest = times[startWin[0]:endWin[4]].reshape(-1, 1)
print(y.shape)
# kernel = RBF(length_scale=1)
kernel = RBF(.01) * ConstantKernel()
gp = GaussianProcessRegressor(kernel=kernel)
gp.fit(X,y)
mu, std = gp.predict(Xtest, return_std=True)
# print(y)
# print(mu)
# print((y-mu))
# print(cov)

plt.plot(X, y, 'bs', ms=8)
# plt.plot(X, f_post)
plt.gca().fill_between(Xtest.flat, mu-2*std, mu+2*std, color="#dddddd")
plt.plot(Xtest, mu, 'r--', lw=2)
# plt.axis([-1, 1, -1, 1])
plt.title('Three samples from the GP posterior')
plt.show()

In [ ]:
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import DotProduct, WhiteKernel
from sklearn.gaussian_process.kernels import ConstantKernel, RBF
data = np.array((normalize(frames[:,:,2]), \
       normalize(frames[:,:,0]), \
       normalize(frames[:,:,1]) )).reshape(1030,53,3)
print(data.shape)
print(data[0,0,:])
gpList = list()
for i in range(len(startWin)):
    y = data[startWin[i]:endWin[i], 0, 0]
    X = times[startWin[i]:endWin[i]].reshape(-1, 1)
#     kernel = ConstantKernel(constant_value=1.0, constant_value_bounds=(0.0, 10.0)) * RBF(length_scale=0.5, length_scale_bounds=(0.0, 10.0)) + RBF(length_scale=2.0, length_scale_bounds=(0.0, 10.0))
    kernel = DotProduct() + WhiteKernel()
    gpr = GaussianProcessRegressor(kernel=kernel,random_state=0).fit(X, y)
    gpList.append(gpr)


In [ ]:
y = (data[:, 0, 0])
X = times.reshape(-1,1)
print(X[:5])
print(y[:5])
print('starting')
for gp in gpList:
#     print(gp.score(X[:10], y[:10]))
    expected, std = gp.predict(X[:2], return_std=True)
#     print(expected-y[:2])
    print(std)

In [ ]:
y[:2]

In [ ]:
params = kernel.get_params()
for key in sorted(params): print("%s : %s" % (key, params[key]))


In [ ]:
print(kernel.theta)  # Note: log-transformed
print(kernel.bounds)

In [ ]:
print(frames.shape)